# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']
R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']
Arxiv has 44 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2408.06491


extracting tarball to tmp_2408.06491... done.
Retrieving document from  https://arxiv.org/e-print/2408.06744


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'gwskynet_pipeline.bbl' from 'tmp_2408.06491/gwskynet_pipeline.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


extracting tarball to tmp_2408.06744...

 done.


R. v. Boekel  ->  R. v. Boekel  |  ['R. v. Boekel']


Found 47 bibliographic references in tmp_2408.06744/ms.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.06744-b31b1b.svg)](https://arxiv.org/abs/2408.06744) | **ASASSN-21js: A multi-year transit of a ringed disc**  |
|| T. H. Pramono, M. A. Kenworthy, <mark>R. v. Boekel</mark> |
|*Appeared on*| *2024-08-14*|
|*Comments*| *9 pages, 4 figures, 6 tables, repository for paper at: this https URL*|
|**Abstract**|            The early-type star ASASSN-21js started to fade in 2021, as was detected by the All Sky Automated Survey for Supernovae, undergoing a multi-year eclipse that is still underway. We interpret this event as being due to a structured disc of material transiting in front of the star. The disc is in orbit around a substellar object with the mass and luminosity of a brown dwarf or smaller. We want to determine the expected duration and ending date of the eclipse. We modelled a tilted and inclined azimuthally symmetric ring system around an unseen companion and calculated the resulting time-varying light curve as the object transited in front of the star. We made an initial estimate of the ring parameters and used these as inputs to an MCMC algorithm to determine the geometric properties of the rings with associated uncertainties. The model most consistent with the light curve to date is a two-ring system at high inclination with respect to the line of sight that has a semi-major axis of 71.6 stellar radii. With an estimate of the stellar radius, the transverse velocity is around 0.7 km/s, which if bound to the star is an orbit with a semi-major axis of around 13000 au, placing it in the Oort cloud of the parent star. The transit is ongoing and will finish around MJD 61526 (May 1 2027). We encourage the community to continue observing this object in order to understand its properties.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2408.06491-b31b1b.svg)](https://arxiv.org/abs/2408.06491) | **GWSkyNet II : a refined machine learning pipeline for real-time classification of public gravitational wave alerts**  |
|| M. L. Chan, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2024-08-14*|
|*Comments*| **|
|**Abstract**|            Electromagnetic follow-up observations of gravitational wave events offer critical insights and provide significant scientific gain from this new class of astrophysical transients. Accurate identification of gravitational wave candidates and rapid release of sky localization information are crucial for the success of these electromagnetic follow-up observations. However, searches for gravitational wave candidates in real time suffer a non-negligible false alarm rate. By leveraging the sky localization information and other metadata associated with gravitational wave candidates, GWSkyNet, a machine learning classifier developed by Cabero et al. (2020), demonstrated promising accuracy for the identification of the origin of event candidates. We improve the performance of the classifier for LIGO-Virgo-KAGRA's fourth observing run by reviewing and updating the architecture and features used as inputs by the algorithm. We also retrain and fine-tune the classifier with data from the third observing run. To improve the prospect of electromagnetic follow-up observations, we incorporate GWSkyNet into LIGO-Virgo-KAGRA's low-latency infrastructure as an automatic pipeline for the evaluation of gravitational wave alerts in real time. We test the readiness of the algorithm on a LIGO-Virgo-KAGRA mock data challenge campaign. The results show that by thresholding on the GWSkyNet score, noise masquerading as astrophysical sources can be rejected efficiently and the majority of true astrophysical signals correctly identified.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2408.06744/./figures/ASASSN-21js_star_model_Teff_14800K.png', 'tmp_2408.06744/./figures/model_combine_240124_bin9days.png', 'tmp_2408.06744/./figures/corner_mcmc_20240208_backup_2e6.png']
copying  tmp_2408.06744/./figures/ASASSN-21js_star_model_Teff_14800K.png to _build/html/
copying  tmp_2408.06744/./figures/model_combine_240124_bin9days.png to _build/html/
copying  tmp_2408.06744/./figures/corner_mcmc_20240208_backup_2e6.png to _build/html/
exported in  _build/html/2408.06744.md
    + _build/html/tmp_2408.06744/./figures/ASASSN-21js_star_model_Teff_14800K.png
    + _build/html/tmp_2408.06744/./figures/model_combine_240124_bin9days.png
    + _build/html/tmp_2408.06744/./figures/corner_mcmc_20240208_backup_2e6.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kms}{km~s^{-1}}$
$\newcommand{\mum}{\mum}$
$\newcommand{\Msun}{M_\odot}$
$\newcommand{\Teff}{T_{\rm{eff}}}$
$\newcommand{\Av}{A_{\rm{V}}}$
$\newcommand{\arraystretch}{1.5}$</div>



<div id="title">

# ASASSN-21js: A multi-year transit of a ringed disc

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2408.06744-b31b1b.svg)](https://arxiv.org/abs/2408.06744)<mark>Appeared on: 2024-08-14</mark> -  _9 pages, 4 figures, 6 tables, repository for paper at: this https URL_

</div>
<div id="authors">

T. H. Pramono, M. A. Kenworthy, <mark>R. v. Boekel</mark>

</div>
<div id="abstract">

**Abstract:**            The early-type star ASASSN-21js started to fade in 2021, as was detected by the All Sky Automated Survey for Supernovae, undergoing a multi-year eclipse that is still underway. We interpret this event as being due to a structured disc of material transiting in front of the star. The disc is in orbit around a substellar object with the mass and luminosity of a brown dwarf or smaller. We want to determine the expected duration and ending date of the eclipse. We modelled a tilted and inclined azimuthally symmetric ring system around an unseen companion and calculated the resulting time-varying light curve as the object transited in front of the star. We made an initial estimate of the ring parameters and used these as inputs to an MCMC algorithm to determine the geometric properties of the rings with associated uncertainties. The model most consistent with the light curve to date is a two-ring system at high inclination with respect to the line of sight that has a semi-major axis of 71.6 stellar radii. With an estimate of the stellar radius, the transverse velocity is around 0.7 km/s, which if bound to the star is an orbit with a semi-major axis of around 13000 au, placing it in the Oort cloud of the parent star. The transit is ongoing and will finish around MJD 61526 (May 1 2027). We encourage the community to continue observing this object in order to understand its properties.         

</div>

<div id="div_fig1">

<img src="tmp_2408.06744/./figures/ASASSN-21js_star_model_Teff_14800K.png" alt="Fig1" width="100%"/>

**Figure 1. -** SED fit to the literature photometry and spectroscopy of ASASSN-21js. Data taken from (1) [Drew, Gonzalez-Solares and Greimel (2014)](), (2) [Evans, Irwin and Helmer (2002)](), (3) [Skrutskie, Cutri and Stiening (2006)](), (4) [Benjamin, Churchwell and Babler (2003)](), [Churchwell, Babler and Meade (2009)](), (5) \citet[][submitted to PASA]{2024arXiv240202015O}, (6) [Vallenari and Brown (2023)](). (*fig:stellar_SED_fit*)

</div>
<div id="div_fig2">

<img src="tmp_2408.06744/./figures/model_combine_240124_bin9days.png" alt="Fig2" width="100%"/>

**Figure 2. -** Plot of different models fitted to the data, which has been binned to 9 day intervals. (*fig:model_combine*)

</div>
<div id="div_fig3">

<img src="tmp_2408.06744/./figures/corner_mcmc_20240208_backup_2e6.png" alt="Fig3" width="100%"/>

**Figure 3. -** Corner plot for the two ring model system. (*fig:two_ring_corner*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2408.06744"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

161  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

9  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
